In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [2]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [0]:
import numpy as np
import pandas as pd
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold
import eli5
from eli5.sklearn import PermutationImportance

In [31]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Feature Engineering

In [0]:
SUFFIX_CUT = '__cut'
for feat in df.columns:
  if isinstance(df[feat][0], list):
    continue
  
  factorized_value = df[feat].factorize()[0]
  if SUFFIX_CUT in feat:
    df[feat] = factorized_value
  else:
    df[feat + SUFFIX_CUT] = factorized_value

In [33]:
cat_feats = [x for x in df.columns if SUFFIX_CUT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

## Decision Tree

In [9]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

## Random Forest

In [10]:
run_model(RandomForestRegressor(n_estimators=50, max_depth=5, random_state=0), cat_feats)

(-18718.657185256638, 64.5424578125788)

## XGBoost

In [11]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}
run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[06:00:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:01:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:01:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [12]:
m = xgb.XGBRegressor(max_depth=5, learning_rate=0.1, n_estimators=50, seed=0)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

[06:03:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd__cut
0.1132 ± 0.0032,param_rok-produkcji__cut
0.1090 ± 0.0025,param_stan__cut
0.0619 ± 0.0025,param_skrzynia-biegów__cut
0.0568 ± 0.0016,param_faktura-vat__cut
0.0489 ± 0.0014,param_moc__cut
0.0273 ± 0.0007,param_marka-pojazdu__cut
0.0242 ± 0.0013,feature_kamera-cofania__cut
0.0212 ± 0.0008,param_typ__cut
0.0174 ± 0.0008,param_pojemność-skokowa__cut


In [0]:
feats = ['param_napęd__cut',
'param_rok-produkcji',
'param_stan__cut',
'param_skrzynia-biegów__cut',
'param_faktura-vat__cut',
'param_moc',
'param_marka-pojazdu__cut',
'feature_kamera-cofania__cut',
'param_typ__cut',
'param_pojemność-skokowa',
'seller_name__cut',
'feature_wspomaganie-kierownicy__cut',
'param_model-pojazdu__cut',
'param_wersja__cut',
'param_kod-silnika__cut',
'feature_system-start-stop__cut',
'feature_asystent-pasa-ruchu__cut',
'feature_czujniki-parkowania-przednie__cut',
'feature_łopatki-zmiany-biegów__cut',
'feature_regulowane-zawieszenie__cut']

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ', '')))

In [37]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[06:27:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:27:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:27:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)